In [69]:
# 生成用于分类的数据，并为每个数据点打标签
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import copy

def tag_entry(x,y):
    if x**2+y**2<1:
        tag=0
    else:
        tag=1
    return tag

def create_data(Number_of_Data):
    entry_list=[]
    for i in range (Number_of_Data):
        x=random.uniform(-2,2)
        y=random.uniform(-2,2)
        tag=tag_entry(x,y)
        entry_list.append([x,y,tag])
    return np.array(entry_list)


In [70]:
#可视化
def plot_data(data,title):
    colors=[]
    for i in data[:,2]:
        if i==0:
            colors.append("orange")
        else:
            colors.append("blue")
    plt.scatter(data[:,0],data[:,1],c=colors)
    plt.title(title)
    plt.show()


In [71]:
def activation_ReLU(inputs):
    return np.maximum(0,inputs)

def activation_softmax(inputs):
    max_value=np.max(inputs,axis=1,keepdims=True)
    slided_inputs=inputs-max_value
    exp_values=np.exp(slided_inputs)
    norm_base=np.sum(exp_values,axis=1,keepdims=True)
    norm_values=exp_values/norm_base
    return norm_values

In [ ]:
class Layer:

    def __init__(self,n_inputs,n_neurons):
        self.weights=np.random.randn(n_inputs,n_neurons)  
        self.biases=np.random.randn(n_neurons)  

    def forward(self,inputs):
        self.sum=np.dot(inputs,self.weights)+self.biases  
        return self.sum  

    def get_weights_adjust_matrix(self,preWeights_values,aftWeights_demands):
        plain_weights=np.full(self.weights.shape,1)
        weight_adjust_matrix=np.full(self.weights.shape,0)
        plain_weights_T=plain_weights.T
        for i in range(BATCH_SIZE):
            weight_adjust_matrix+=(plain_weights_T*preWeights_values[i,:]).T*aftWeights_demands[i,:]
        weight_adjust_matrix/=BATCH_SIZE
        return weight_adjust_matrix

返回的是权重梯度矩阵
函数返回的是每个权重需要调整的梯度值，不是新的权重矩阵本身。

这基于反向传播的链式法则：
权重梯度公式：$$ ∂Loss/∂W[i,j] = ∂Loss/∂output[j] × ∂output[j]/∂W[i,j]$$
其中：
$$∂Loss/∂output[j] = aftWeights_demands[j] (当前层第j个神经元的误差)$$
$$∂output[j]/∂W[i,j] = preWeights_values[i] (前一层第i个神经元的输出)$$

直觉理解
前一层输出越大 → 该权重对最终结果影响越大 → 梯度越大
当前层需求越大 → 该神经元误差越大 → 对应权重需要更大调整
所以 前一层输出 × 当前层需求 = 权重梯度
这个梯度告诉我们权重应该往哪个方向、调整多少，用于后续的权重更新：new_weights = old_weights - learning_rate × 梯度

In [73]:

class Network:
    def __init__(self,network_shape):
        self.shape=network_shape
        self.layers=[]
        for i in range(len(network_shape)-1):
            layer=Layer(network_shape[i],network_shape[i+1])
            self.layers.append(layer)
    #前馈运算
    def network_forward(self,inputs):
        outputs=[inputs]
        for i in range(len(self.layers)):
            layer_output=self.layers[i].forward(outputs[i])
            if i==len(self.layers)-1:
                layer_output=activation_softmax(layer_output)
            else:
                layer_output=activation_ReLU(layer_output)
            outputs.append(layer_output)
        return outputs

In [74]:
#分类函数
def classify(probabilities):
    classification=np.rint(probabilities[:,1])
    return classification

In [75]:
def precise_loss_function(predicted,real):

    # 创建真实标签的one-hot编码矩阵
    real_matrix=np.zeros((len(real),2))  # 使用是len(real)
    real_matrix[:,1]=real.flatten()  # 使用flatten()将(1,5)转换为(5,)，赋值给第二列（正类）
    real_matrix[:,0]=1-real  # 第一列为负类（1-正类标签）
    print("真实标签补全矩阵:\n",real_matrix)
    
    # 计算预测值与真实标签的点积，沿axis=1求和
    product=np.sum(predicted*real_matrix,axis=1)
    
    # 返回损失值：1减去点积
    return 1-product


In [76]:
def get_final_layer_preAct_demands(predicted_value,target_value):
    """
    计算最终层激活前的需求值（demands），用于反向传播的梯度计算
    
    参数:
    predicted_value: 预测概率值，形状为 (m, 2)，每行为一个样本的[负类概率, 正类概率]
    target_value: 真实标签，形状为 (m,)，值为0或1
    
    返回:
    target: 梯度信号向量，形状为 (m, 2)
            - 如果预测正确：返回 [0, 0]（无需调整）
            - 如果预测错误：返回放大的误差向量，指导权重如何调整
    """
    # 创建one-hot编码矩阵
    target=np.zeros((len(target_value),2))
    target[:,1]=target_value  # 第二列为正类标签
    target[:,0]=1-target_value  # 第一列为负类标签（1-正类）
    
    # 遍历每个样本
    for i in range(len(predicted_value)):
        # 计算预测值与真实标签的点积，判断预测是否正确
        if np.dot(target[i],predicted_value[i])>0.5:
            # 预测正确：点积>0.5，设置为零向量（无需调整权重）
            target[i]=np.array([0,0])
        else:
            # 预测错误：计算放大的误差向量
            # (target-0.5)*2 将-0.5~0.5的误差放大到-1~1，增强梯度信号
            target[i]=(target[i]-0.5)*2
    return target
    